In [1]:
import requests
import unicodedata
import re

# ================== Cấu hình ==================
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "qwen2.5:7b-instruct"
DEBUG = False

LABELS = ["very_positive", "positive", "neutral", "negative", "very_negative"]

FEW_SHOT = [
    ('"san pham ban phim co hai long, gia cong on; giao nhanh."', 'positive'),
    ('"Dùng văn phòng thì được , vỏ máy khá"', 'neutral'),
    ('"âm thanh tạm , pin chưa ổn , còn lại không nổi bật"', 'neutral'),
    ('"sản phẩm laptop rất kinh khủng, gia công cực kinh khủng; phản hồi chậm, rất bực mình."', 'very_negative'),
    ('"dien thoai gia cao, nhiet do chua tot; dong goi au."', 'negative'),
    ('"máy tính bảng quá tuyệt vời, dịch vụ nổi bật, hỗ trợ tốt; rất đáng tiền."', 'very_positive'),
    ('"Giao hàng khá nhanh , đóng gói không tốt , chất lượng bình thường"', 'neutral'),
]

SYSTEM_PROMPT = """Bạn là bộ phân loại cảm xúc bình luận sản phẩm tiếng Việt.
Nhãn hợp lệ: very_positive, positive, neutral, negative, very_negative.

Yêu cầu:
- Đọc bình luận sản phẩm.
- Chọn MỘT nhãn duy nhất thể hiện cảm xúc tổng thể.
- CHỈ trả về đúng chuỗi nhãn, KHÔNG thêm bất kỳ văn bản nào khác.

Nhãn trả về phải là MỘT TRONG:
very_positive
positive
neutral
negative
very_negative
"""

def build_prompt(user_text: str) -> str:
    lines = []
    for t, lab in FEW_SHOT:
        lines.append(f'Bình luận: {t}')
        lines.append(f'Nhãn: {lab}')
        lines.append("")
    lines.append(f'Bình luận: "{user_text}"')
    lines.append('Nhãn:')
    return "\n".join(lines)

def call_ollama(prompt: str) -> str:
    payload = {
        "model": MODEL_NAME,
        "prompt": SYSTEM_PROMPT + "\n\n" + prompt,
        "options": {
            "temperature": 0.0,
            "top_p": 1.0,
            "seed": 42,
        },
        # "keep_alive": 0, 
        "stream": False,
    }
    r = requests.post(OLLAMA_URL, json=payload, timeout=120)
    r.raise_for_status()
    resp = r.json()
    raw = resp.get("response", "").strip()

    if DEBUG:
        print("========== RAW MODEL OUTPUT ==========")
        print(raw)
        print("======================================")

    return raw

def normalize_str(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = s.strip().lower()
    return s

def extract_label(raw: str) -> str:
    """Cố gắng lôi ra một nhãn trong LABELS từ string raw."""
    if not raw:
        return "neutral"

    s = normalize_str(raw)

    # 1) Nếu chứa trực tiếp nhãn chuẩn
    for lab in LABELS:
        if lab in s:
            return lab

    # 2) Alias đơn giản (tiếng Việt, viết liền, viết cách)
    alias_map = {
        "very positive": "very_positive",
        "verypositive": "very_positive",
        "rất tốt": "very_positive",
        "tuyệt vời": "very_positive",

        "positive": "positive",
        "tốt": "positive",
        "good": "positive",

        "neutral": "neutral",
        "trung tính": "neutral",
        "bình thường": "neutral",
        "binh thuong": "neutral",
        "ổn": "neutral",
        "on": "neutral",

        "negative": "negative",
        "negativ": "negative",
        "xấu": "negative",
        "bad": "negative",

        "very negative": "very_negative",
        "verynegative": "very_negative",
        "rất tệ": "very_negative",
        "kinh khủng": "very_negative",
        "tồi tệ": "very_negative",
    }

    for key, lab in alias_map.items():
        if key in s:
            return lab

    # 3) Tách token chữ cái/underscore, thử từng cái
    tokens = re.findall(r"[a-z_]+", s)
    for tok in tokens:
        t = normalize_str(tok).replace("-", "_").replace(".", "")
        if t in alias_map:
            return alias_map[t]
        if t in LABELS:
            return t

    return "neutral"

def classify_sentiment(text: str) -> str:
    """Hàm high-level: nhập câu, trả về nhãn."""
    prompt = build_prompt(text)
    raw = call_ollama(prompt)
    label = extract_label(raw)
    return label


In [27]:
sentence = "loa hỏng ngay lần 1, giao hàng kẹt mãi."
print("Text:", sentence)
print("Predicted label:", classify_sentiment(sentence))


Text: loa hỏng ngay lần 1, giao hàng kẹt mãi.
Predicted label: negative


In [26]:
samples = [
    "loa hỏng ngay lần 1, giao hàng kẹt mãi.",
    "Sản phẩm laptop đỉnh cao 🤩, dùng khá ok.",
    "robot hút bụi không nổi bật, giá tạm ổn.",
    "tai nghe không chê vào đâu đc, quá yêu.",
]

for s in samples:
    print(s, "->", classify_sentiment(s))


loa hỏng ngay lần 1, giao hàng kẹt mãi. -> negative
Sản phẩm laptop đỉnh cao 🤩, dùng khá ok. -> positive
robot hút bụi không nổi bật, giá tạm ổn. -> neutral
tai nghe không chê vào đâu đc, quá yêu. -> very_positive
